### Check Hardware & RAM availability:
Commands to check for available GPU and RAM allocation on runtime

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Oct 26 06:37:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### References:
* https://huggingface.co/
* https://arxiv.org/abs/1907.11692

### Install Required Libraries for Transformer Models:
* Pre-Trained Transformer models are part of Hugging Face Library(transformers).
* Similarly, any datatset part of Hugging Face can be called from the datasets library.
* Finally we will use a high level abstraction package called k-train to simplify our modelling and predictions

In [2]:
!pip install ktrain
!pip install transformers
!pip install datasets

     |████████████████████████████████| 25.3 MB 1.1 MB/s 
     |████████████████████████████████| 6.8 MB 23.9 MB/s 
     |████████████████████████████████| 981 kB 65.5 MB/s 
     |████████████████████████████████| 263 kB 77.5 MB/s 
     |████████████████████████████████| 2.8 MB 67.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 468 kB 59.6 MB/s 
     |████████████████████████████████| 3.3 MB 79.0 MB/s 
     |████████████████████████████████| 895 kB 72.6 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 72.5 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.2-py3-none-any.whl size=25290606 sha256=36772e8c41724275b178c3c6a9632ac308349c9f2c6736ead3aa68dbaa684292
  Stored in directory: /root/.cache/pip/wheels/df/2b/04/7c821b51e637ec480060989b5030d0c4cce16efe0d67bff94b
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=0b1c50075c7549c4

### Import Libraries:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ktrain
from ktrain import text
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import list_datasets
from datasets import load_dataset
import timeit
import warnings

pd.set_option('display.max_columns', None)
warnings.simplefilter(action="ignore")

In [4]:
tf.__version__

'2.6.0'

### Load Dataset:

In [5]:
emotion_train = load_dataset('emotion', split='train')
emotion_val = load_dataset('emotion', split='validation')

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


Using custom data configuration default
Reusing dataset emotion (/root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


In [6]:
print("\nTrain Dataset Features for Emotion: \n", emotion_train.features)
print("\nValidation Dataset Features for Emotion: \n", emotion_val.features)


Train Dataset Features for Emotion: 
 {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], names_file=None, id=None)}

Validation Dataset Features for Emotion: 
 {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], names_file=None, id=None)}


In [7]:
emotion_train_df = pd.DataFrame(data=emotion_train)
emotion_val_df = pd.DataFrame(data=emotion_val)

In [8]:
class_label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

### Split Train & Validation data:

In [9]:
X_train = emotion_train_df[:]["text"]
y_train = emotion_train_df[:]["label"]
X_test = emotion_val_df[:]["text"]
y_test = emotion_val_df[:]["label"]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(16000,) (16000,) (2000,) (2000,)


### Instantiating a DistilBERT Instance:
Create a DistilBERT instance with the model name, max token length, the labels to be used for each category and the batch size.

In [10]:
distilbert_transformer = text.Transformer('distilbert-base-uncased', maxlen=512, classes=class_label_names, batch_size=6)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

### Perform Data Preprocessing:

In [11]:
distilbert_train = distilbert_transformer.preprocess_train(X_train.to_list(), y_train.to_list())
distilbert_val = distilbert_transformer.preprocess_test(X_test.to_list(), y_test.to_list())

preprocessing train...
language: en
train sequence lengths:
	mean : 19
	95percentile : 41
	99percentile : 52


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 19
	95percentile : 40
	99percentile : 52


### Compile AlBERT in a K-Train Learner Object:
Since we are using k-train as a high level abstration package, we need to wrap our model in a k-train Learner Object for further compuation

In [12]:
distilbert_model = distilbert_transformer.get_classifier()

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [13]:
distilbert_learner_ins = ktrain.get_learner(model=distilbert_model,
                            train_data=distilbert_train,
                            val_data=distilbert_val,
                            batch_size=6)

### BERT Model Summary:

In [14]:
distilbert_learner_ins.model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  4614      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,958,086
Trainable params: 66,958,086
Non-trainable params: 0
_________________________________________________________________


### BERT Optimal Learning Rates:¶
BERT follows Knowledge Distillation on BERT, hence we can use the established batch sizes and learning rates as used in BERT:

* Batch Sizes => {16, 32}
* Learning Rates => {1e−5, 2e−5, 3e−5}
We will choose the maximum among these for our fine-tuning and evaluation purposes.

### Fine Tuning DistilBERT on Emotion Dataset:
We take our Dbpedia Ontology dataset along with the BERT model we created, define the learning-rate & epochs to be used and start fine-tuning.

In [15]:
distilbert_fine_tuning_start= timeit.default_timer()
distilbert_learner_ins.fit_onecycle(lr=2e-5, epochs=3)
distilbert_fine_tuning_stop = timeit.default_timer()



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
2667/2667 [==============================] - 611s 225ms/step - loss: 0.7052 - accuracy: 0.7559 - val_loss: 0.2211 - val_accuracy: 0.9225
Epoch 2/3
2667/2667 [==============================] - 602s 225ms/step - loss: 0.1846 - accuracy: 0.9273 - val_loss: 0.1409 - val_accuracy: 0.9360
Epoch 3/3
2667/2667 [==============================] - 603s 225ms/step - loss: 0.1017 - accuracy: 0.9511 - val_loss: 0.1286 - val_accuracy: 0.9390


In [16]:
print("\nFine-Tuning time for DistilBERT on Emotion dataset: \n", (distilbert_fine_tuning_stop - distilbert_fine_tuning_start)/60, " min")


Fine-Tuning time for DistilBERT on Emotion dataset: 
 30.270395053433333  min


### Checking DistilBERT performance metrics:

In [17]:
distilbert_validation_start= timeit.default_timer()
distilbert_learner_ins.validate()
distilbert_validation_stop= timeit.default_timer()

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       550
           1       0.97      0.95      0.96       704
           2       0.84      0.93      0.88       178
           3       0.98      0.89      0.93       275
           4       0.89      0.92      0.90       212
           5       0.90      0.79      0.84        81

    accuracy                           0.94      2000
   macro avg       0.92      0.91      0.91      2000
weighted avg       0.94      0.94      0.94      2000



In [18]:
print("\nInference time for DistilBERT on Emotion dataset: \n", (distilbert_validation_stop - distilbert_validation_start), " sec")


Inference time for DistilBERT on Emotion dataset: 
 4.459392552999816  sec


In [19]:
distilbert_learner_ins.validate(class_names=class_label_names)

              precision    recall  f1-score   support

     sadness       0.95      0.98      0.96       550
         joy       0.97      0.95      0.96       704
        love       0.84      0.93      0.88       178
       anger       0.98      0.89      0.93       275
        fear       0.89      0.92      0.90       212
    surprise       0.90      0.79      0.84        81

    accuracy                           0.94      2000
   macro avg       0.92      0.91      0.91      2000
weighted avg       0.94      0.94      0.94      2000



array([[541,   1,   1,   1,   6,   0],
       [  2, 669,  29,   1,   1,   2],
       [  1,  12, 165,   0,   0,   0],
       [ 18,   3,   1, 245,   8,   0],
       [  9,   1,   0,   3, 194,   5],
       [  1,   6,   0,   0,  10,  64]])

In [20]:
distilbert_learner_ins.view_top_losses(preproc=distilbert_transformer)

----------
id:975 | loss:7.29 | true:love | pred:joy)

----------
id:1304 | loss:5.98 | true:surprise | pred:joy)

----------
id:177 | loss:5.73 | true:sadness | pred:joy)

----------
id:1111 | loss:4.35 | true:joy | pred:anger)



### Saving DistilBERT Model:

In [21]:
distilbert_predictor = ktrain.get_predictor(distilbert_learner_ins.model, preproc=distilbert_transformer)
distilbert_predictor.get_classes()

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [23]:
distilbert_predictor.save('/content/distilbert-predictor-on-emotion')

In [24]:
!zip -r /content/distilbert-predictor-on-emotion /content/distilbert-predictor-on-emotion

  adding: content/distilbert-predictor-on-emotion/ (stored 0%)
  adding: content/distilbert-predictor-on-emotion/tf_model.preproc (deflated 46%)
  adding: content/distilbert-predictor-on-emotion/tokenizer_config.json (deflated 40%)
  adding: content/distilbert-predictor-on-emotion/config.json (deflated 52%)
  adding: content/distilbert-predictor-on-emotion/special_tokens_map.json (deflated 40%)
  adding: content/distilbert-predictor-on-emotion/tf_model.h5 (deflated 8%)
  adding: content/distilbert-predictor-on-emotion/vocab.txt (deflated 53%)
  adding: content/distilbert-predictor-on-emotion/tokenizer.json (deflated 59%)
